# 10 notebook that applies all project cards (replacing notebooks 02-09)

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy
import re

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-09-30 12:01:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 12:01:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 12:01:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 12:01:19, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [6]:
parameters.zones

4756

# Read pickle

In [7]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

In [8]:
v_01_scenario.transit_net.feed.stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21881 entries, 0 to 21880
Data columns (total 20 columns):
stop_name              21881 non-null object
stop_lat               21881 non-null float64
stop_lon               21881 non-null float64
zone_id                8233 non-null object
agency_raw_name        21881 non-null object
stop_code              15289 non-null object
location_type          5475 non-null float64
parent_station         249 non-null object
stop_desc              409 non-null object
stop_url               261 non-null object
stop_timezone          126 non-null object
wheelchair_boarding    377 non-null float64
platform_code          52 non-null object
position               0 non-null object
direction              0 non-null object
* used by routes       124 non-null object
stop_id                21881 non-null object
osm_node_id            21212 non-null object
shst_node_id           21212 non-null object
model_node_id          21881 non-null object
dtypes: floa

In [9]:
v_01_scenario.transit_net.feed.stops[
    v_01_scenario.transit_net.feed.stops.stop_name == "San Jose"]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
5466,San Jose,37.329935,-121.902496,56146,Capitol_2017_3_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6680,NaN,NaN,2192825


# Create scenario and apply projects

waiting on dependencies to work

In [10]:
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df["shstGeometryId"]

In [11]:
step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    tags = ['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only', 'Toll Plaza'],
    validate_project_cards=False)

2021-09-30 12:01:58, INFO: Creating Scenario
2021-09-30 12:01:58, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [12]:
step1_scenario.apply_all_projects()

2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2021-09-30 12:02:03, INFO: Applying Project 

C:\Users\wangs1\AppData\Roaming\Python\Python37\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-09-30 12:02:04, INFO: Applying Project to Transit Network: add san mateo bridge toll plaza
2021-09-30 12:02:04, INFO: Applying Project to Transit Network: add san mateo bridge toll plaza
2021-09-30 12:02:04, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-09-30 12:02:04, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-09-30 12:02:04, INFO: Applying add san mateo bridge toll plaza
2021-09-30 12:02:04, INFO: Applying add san mateo bridge toll plaza
2021-09-30 12:02:04, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza
2021-09-30 12:02:04, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-09-30 12:03:58, INFO: Applying Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-09-30 12:03:58, INFO: Applying Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-09-30 12:03:58, INFO: Applying Year 2015 Modify Facilit

2021-09-30 12:04:01, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Project to Roadway Network: Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Project to Roadway Network: Year 2015 Should be 5 Lane correct facility type
2021-09-30 12:04:01, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-09-30 12:04:01, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-09-30 12:04:01, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-09-30 12:04:01, INFO: Applying Year 2015 Modify Lanes to 5 Lanes
2021-09-30 12:04:01, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 5 Lanes
2021-09-30 12:04:01, INFO: Applying Project to Roadway N

2021-09-30 12:04:04, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:04, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:04, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:04, INFO: Applying Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:04, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:04, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 1 Lane
2021-09-30 12:04:05, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 14 Lanes
2021-09-30 12:04:05, INFO: Applying Year 2015 Sh

2021-09-30 12:07:52, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-09-30 12:07:52, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-09-30 12:07:52, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-09-30 12:07:52, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-09-30 12:07:52, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2021-09-30 12:07:52, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-09-30 12:07:52, INFO: Applying year 2015 sterling on ramp HOV only in PM
2021-09-30 12:07:52, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-09-30 12:07:52, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2021-09-30 12:07:52, INFO: Applying year 2015 golden gate reversible lanes
2021-09-30 

In [13]:
step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198][["A", "B", "geometry", "id"]]

,A,B,geometry,id
1634794,5013515,5030001,"LINESTRING (-122.48221 37.83307, -122.48175 37...",d01b27cabff869e0bf4f7187ee77d6de


In [14]:
step1_scenario.road_net.nodes_df[step1_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1,0,0,5030001,0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [15]:
list(step1_scenario.road_net.links_df[step1_scenario.road_net.links_df.model_link_id == 8063198].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [16]:
list(step1_scenario.road_net.shapes_df[step1_scenario.road_net.shapes_df.id == "d01b27cabff869e0bf4f7187ee77d6de"].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [17]:
step2_scenario = Scenario.create_scenario(
    base_scenario = step1_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate_project_cards=False)

2021-09-30 12:07:55, INFO: Creating Scenario
2021-09-30 12:07:55, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [18]:
step2_scenario.apply_all_projects()

2021-09-30 12:07:59, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:07:59, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:07:59, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:07:59, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:07:59, INFO: Applying year 2015 modify hov lanes and use on san mateo bridge t

C:\Users\wangs1\AppData\Roaming\Python\Python37\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-09-30 12:08:00, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-09-30 12:08:00, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-09-30 12:08:00, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-09-30 12:08:00, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-09-30 12:08:00, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2021-09-30 12:08:01, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2021-09-30 12:08:01, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2021-09-30 12:08:01, INFO: Applying Year 2015 Modify HOV 

2021-09-30 12:08:39, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-09-30 12:08:39, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-09-30 12:08:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-09-30 12:08:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-09-30 12:08:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-09-30 12:08:47, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-09-30 12:08:47, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2021-09-30 12:08:47, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expres

2021-09-30 12:09:39, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:39, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:39, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:39, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:39, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-09-30 12:09:40, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-09-30 12:09:40, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-09-30 12:09:40, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge 

2021-09-30 12:10:07, INFO: Applying Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-09-30 12:10:07, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-09-30 12:10:07, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Springtown Blvd to Isabel Ave
2021-09-30 12:10:08, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:08, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:08, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:08, INFO: Applying Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:08, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:08, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-580 WB Isabel Ave to Fallon Road
2021-09-30 12:10:09, 

2021-09-30 12:10:14, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-09-30 12:10:14, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-09-30 12:10:14, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2021-09-30 12:10:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-09-30 12:10:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-09-30 12:10:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-09-30 12:10:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2021-09-30 12:10:17, INFO: Applying Proje

2021-09-30 12:10:55, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-09-30 12:10:55, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-09-30 12:10:57, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-09-30 12:10:57, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-09-30 12:10:57, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-09-30 12:10:57, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-09-30 12:10:57, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2021-09-30 12:10:57, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 10

2021-09-30 12:11:02, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Dumbarton Bridge Toll Plaza Toll Plaza on CA-84 WB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Dumbarton Bridge Toll Plaza Toll Plaza on CA-84 WB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
2021-09-30 12:11:02, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Carquinez Bridge Toll Plaza on I-80 NB - Bridge Pass
202

In [19]:
step2_scenario.road_net.links_df.shape

(1634796, 44)

In [20]:
step2_scenario.road_net.links_df.model_link_id.nunique()

1634796

In [21]:
step2_scenario.road_net.links_df[step2_scenario.road_net.links_df.model_link_id == 3149471].useclass

762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [22]:
step2_scenario.road_net.links_df.county.isnull().sum()

0

In [23]:
step2_scenario.road_net.links_df.model_link_id.max()

8063199

In [24]:
step3_scenario = Scenario.create_scenario(
    base_scenario= step2_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Major Transit links"],
    validate_project_cards=False)

step3_scenario.apply_all_projects()

2021-09-30 12:11:04, INFO: Creating Scenario
2021-09-30 12:11:04, INFO: Creating Scenario
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:11:09, INFO: Applying Project to Roadway Network: move street rail to street connector links


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-09-30 12:12:50, INFO: Applying move street rail to street connector links
2021-09-30 12:12:50, INFO: Applying move street rail to street connector links
2021-09-30 12:12:50, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:12:50, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:14:30, INFO: Applying move street rail to street connector links
2021-09-30 12:14:30, INFO: Applying move street rail to street connector links
2021-09-30 12:14:30, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:14:30, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-09-30 12:16:11, INFO: Applying move street rail to street connector links
2021-09-30 12:16:11, INFO: Applying move street rail to street connector links
2021-09-30 12:16:11, INFO: Applying Project to Roadway Network: move street rail to street connect

In [25]:
step3_scenario.road_net.links_df.shape

(1634814, 44)

In [26]:
step3_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'geometry', 'ft_cal', 'ft', 'useclass',
       'assignable_cal', 'assignable', 'transit', 'id', 'tollbooth',
       'bus_only', 'transit_access', 'managed', 'ML_lanes', 'segment_id',
       'ML_tollbooth', 'ML_useclass', 'ML_access', 'ML_egress', 'ML_tollseg',
       'tollseg'],
      dtype='object')

In [27]:
step3_scenario.transit_net.set_roadnet(step3_scenario.road_net, 
                                      validate_consistency = False)

step4_scenario = Scenario.create_scenario(
    base_scenario= step3_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Minor Transit", "Add Transit", "Major Transit", "Toll Plaza Transit"],
    validate_project_cards=False)

step4_scenario.apply_all_projects()

2021-09-30 12:45:39, INFO: Creating Scenario
2021-09-30 12:45:39, INFO: Creating Scenario
2021-09-30 12:48:51, ERROR: Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-09-30 12:48:51, ERROR: Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-09-30 12:48:51, ERROR: Projects update routing for golden gate bridge toll plaza transit has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:48:51, ERROR: Projects update routing for golden gate bridge toll plaza transit has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:48:51, ERROR: Projects update routing for san mateo bridge toll plaza transit has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2021-09-30 12:48:51,

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-09-30 12:49:11, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-09-30 12:49:12, INFO: Applying VTA Route 22
2021-09-30 12:49:12, INFO: Applying VTA Route 22
2021-09-30 12:49:12, INFO: Applying Project to Transit Network: VTA Route 22
2021-09-30 12:49:12, INFO: Applying Project to Transit Network: VTA Route 22
2021-09-30 12:49:13, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing, find complete path using default graph
2021-09-30 12:49:13, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing, find complete

2021-09-30 12:49:15, INFO: Creating a new stop in stops.txt for node ID: 2046425
2021-09-30 12:49:15, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-09-30 12:49:15, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2038694
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2038694
2021-09-30 12:49:16, INFO: Creating a new stop in stops.txt for node ID: 2003121
2021-09-30 12:49:16, INFO: C

2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2002834
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2163919
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2163919
2021-09-30 12:49:21, INFO: Creating a new stop in stops.txt for node ID: 2059787
2021-09-30 12:49:21, INFO: C

...

2021-09-30 12:51:25, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:25, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:25, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:25, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:25, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-09-30 12:51:25, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-09-30 12:51:26, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:26, INFO: Applying SFMTA Route 346 Edits
2021-09-30 12:51:26, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-09-30 12:51:26, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-09-30 12:51:27, INFO: Creating a new stop in stops.txt for node ID: 1018498
2021-09-30 12:51:27, INFO: Creating a new stop in stops.txt for node ID: 1018498
2021-09-30 12:51:27, INFO: Creating a new stop in stops.txt for node ID: 1022481
2021-09-30 12:51:27, INFO: Creating a new stop in stops.txt for n

2021-09-30 12:51:51, INFO: Creating a new stop in stops.txt for node ID: 1012353
2021-09-30 12:51:51, INFO: Creating a new stop in stops.txt for node ID: 1012353
2021-09-30 12:51:51, INFO: Creating a new stop in stops.txt for node ID: 1020811
2021-09-30 12:51:51, INFO: Creating a new stop in stops.txt for node ID: 1020811
2021-09-30 12:51:52, INFO: Applying SFMTA Route 306 Edits
2021-09-30 12:51:52, INFO: Applying SFMTA Route 306 Edits
2021-09-30 12:51:52, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-09-30 12:51:52, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-09-30 12:51:53, INFO: Applying SFMTA Route 306 Edits
2021-09-30 12:51:53, INFO: Applying SFMTA Route 306 Edits
2021-09-30 12:51:53, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-09-30 12:51:53, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-09-30 12:51:54, INFO: Applying SFMTA Route 306 Edits
2021-09-30 12:51:54, INFO: Applying SFMTA 

2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1513976
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1528479
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1528479
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1529690
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1529690
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1529454
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1529454
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1519622
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1519622
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1513122
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1513122
2021-09-30 13:05:29, INFO: Creating a new stop in stops.txt for node ID: 1514054
2021-09-30 13:05:29, INFO: C

2021-09-30 13:05:37, INFO: Applying Project to Transit Network: BART Fremont Daly City reverse shape
2021-09-30 13:05:38, INFO: Applying year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Applying year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Applying year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Applying year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Applying Project to Transit Network: year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Applying Project to Transit Network: year 2015 ac transit route 1r
2021-09-30 13:05:38, INFO: Creating a new stop in stops.txt for node ID: 2502166
2021-09-30 13:05:38, INFO: Creating a new stop in stops.txt for node ID: 2502166
2021-09-30 13:05:38, INFO: Creating a new stop in stops.txt for node ID: 2503906
2021-09-30 13:05:38, INFO: Creating a new stop in stops.txt for node ID: 2503906
2021-09-30 13:05:39, INFO: Applying year 2015 ac transit route 1
2021-09-30 13:05:39, INFO: Applying year 2015 ac transit r

2021-09-30 13:05:50, INFO: Creating a new stop in stops.txt for node ID: 2503531
2021-09-30 13:05:50, INFO: Creating a new stop in stops.txt for node ID: 2539453
2021-09-30 13:05:50, INFO: Creating a new stop in stops.txt for node ID: 2539453
2021-09-30 13:05:50, INFO: Creating a new stop in stops.txt for node ID: 2564896
2021-09-30 13:05:50, INFO: Creating a new stop in stops.txt for node ID: 2564896
2021-09-30 13:05:51, INFO: Applying year 2015 ac transit route 06 edits
2021-09-30 13:05:51, INFO: Applying year 2015 ac transit route 06 edits
2021-09-30 13:05:51, INFO: Applying year 2015 ac transit route 06 edits
2021-09-30 13:05:51, INFO: Applying year 2015 ac transit route 06 edits
2021-09-30 13:05:51, INFO: Applying Project to Transit Network: year 2015 ac transit route 06 edits
2021-09-30 13:05:51, INFO: Applying Project to Transit Network: year 2015 ac transit route 06 edits
2021-09-30 13:05:52, INFO: Creating a new stop in stops.txt for node ID: 2592196
2021-09-30 13:05:52, INFO:

In [28]:
step4_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = step4_scenario.road_net,
    parameters = parameters)

2021-09-30 13:05:53, INFO: Adding centroid and centroid connector to standard network
2021-09-30 13:05:53, INFO: Adding centroid and centroid connector to standard network
2021-09-30 13:05:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 13:05:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 13:05:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 13:05:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 13:06:08, INFO: Finished adding centroid and centroid connectors
2021-09-30 13:06:08, INFO: Finished adding centroid and centroid connectors


In [29]:
print(step4_scenario.road_net.links_df.shape)
print(step4_scenario.road_net.links_df.model_link_id.nunique())
print(step4_scenario.road_net.nodes_df.shape)
print(step4_scenario.road_net.nodes_df.model_node_id.nunique())
print(step4_scenario.road_net.shapes_df.shape)

(2199316, 44)
2199316
(688964, 12)
688964
(1152265, 6)


In [30]:
step4_scenario.road_net.links_df.locationReferences.iloc[-1]

[{'sequence': 1,
  'point': [-122.50580293859502, 37.94830473568058],
  'distanceToNextRef': 51.382886808196375,
  'bearing': 0,
  'intersectionId': nan},
 {'sequence': 2,
  'point': [-122.50576480000001, 37.9487303],
  'intersectionId': '44c53879b5a9797cbb2eff72b5f64699'}]

In [31]:
step5_scenario = Scenario.create_scenario(
    base_scenario= step4_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['External Stations Review'],
    validate_project_cards=False)

step5_scenario.apply_all_projects()

2021-09-30 13:06:09, INFO: Creating Scenario
2021-09-30 13:06:09, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


2021-09-30 13:09:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-09-30 13:09:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-09-30 13:09:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-09-30 13:09:38, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-09-30 13:09:38, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-09-30 13:09:38, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-09-30 13:11:52, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:11:52, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:11:52, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:11:52, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:11:52, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:11:52, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-09-30 13:14:06, INFO: Applying Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-1

2021-09-30 13:34:21, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:34:21, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:34:21, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:34:21, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:34:21, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:34:21, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-09-30 13:36:34, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 SB
2021-09-30 13:36:34, INFO: Applying Year 2015 Add centroi

2021-09-30 13:56:43, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-09-30 13:56:43, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-09-30 13:56:43, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-09-30 13:56:43, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-09-30 13:58:58, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-09-30 13:58:58, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-09-30 13:58:58, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-09-30 13:58:58, INFO

In [32]:
print(step5_scenario.road_net.links_df.shape)
print(step5_scenario.road_net.links_df.model_link_id.nunique())
print(step5_scenario.road_net.nodes_df.shape)
print(step5_scenario.road_net.nodes_df.model_node_id.nunique())

(2199342, 44)
2199342
(688964, 12)
688964


In [33]:
# add taps and tap connectors
step5_scenario.road_net = mtc.add_tap_and_tap_connector(
    roadway_network = step5_scenario.road_net,
    transit_network = step5_scenario.transit_net, 
    parameters = parameters)

2021-09-30 14:08:27, INFO: Adding tap and tap connector to standard network
2021-09-30 14:08:27, INFO: Adding tap and tap connector to standard network
2021-09-30 14:08:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:08:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:08:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:08:27, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:08:29, INFO: There are 601 stops not connected to taps, connecting them to the closest taps.
2021-09-30 14:08:29, INFO: There are 601 stops not connected to taps, connecting them to the closest taps.
snapping gtfs stops to roadway node osmid...


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2592: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  crs = "EPSG:4326"


2021-09-30 14:08:36, INFO: Finished adding tap and tap connectors
2021-09-30 14:08:36, INFO: Finished adding tap and tap connectors


In [36]:
# add county

step5_scenario.road_net = mtc.calculate_county(
    roadway_network = step5_scenario.road_net,
    parameters = parameters,
    network_variable = 'county'
)

2021-09-30 14:10:16, INFO: County Variable 'county' already in network, calculating for missing ones
2021-09-30 14:10:16, INFO: County Variable 'county' already in network, calculating for missing ones
2021-09-30 14:10:16, INFO: Adding roadway network variable for county using a spatial join with: Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\county\county.shp
2021-09-30 14:10:16, INFO: Adding roadway network variable for county using a spatial join with: Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\county\county.shp


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2759: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid


2021-09-30 14:13:28, INFO: Finished Calculating link county variable: county
2021-09-30 14:13:28, INFO: Finished Calculating link county variable: county
2021-09-30 14:14:23, INFO: Finished Calculating node county variable: county
2021-09-30 14:14:23, INFO: Finished Calculating node county variable: county


In [37]:
step5_scenario.road_net.links_df.county.value_counts()

Santa Clara      660008
Alameda          443532
Contra Costa     326621
San Mateo        203882
Sonoma           178792
Solano           157454
San Francisco    133512
Marin             84883
Napa              47068
External             28
Name: county, dtype: int64

In [38]:
step5_scenario.road_net.nodes_df.county.value_counts()

Santa Clara      203748
Alameda          137128
Contra Costa     104665
San Mateo         61887
Sonoma            59970
Solano            50710
San Francisco     33406
Marin             28439
Napa              14990
External             21
Name: county, dtype: int64

# sanity check

In [39]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
765902,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
548523,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 4, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN


In [41]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2199322, dtype: object

In [42]:
# this link is deleted and splitted into two for toll plaza at GGB

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8016300]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg


In [43]:
# golden gate bridge

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].lanes.iloc[0]

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [44]:
# golden gate bridge HOV

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].ML_lanes.iloc[0]

{'default': 0,
 'timeofday': [{'time': (21600, 36000), 'value': 2},
  {'time': (54000, 68400), 'value': 2}]}

In [45]:
# check if new node is created for toll plaza at GGB

step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1.0,0.0,0.0,5030001,0.0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [46]:
# golden gate bridge toll plaza

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].lanes.iloc[0]

{'default': 9,
 'timeofday': [{'time': (21600, 36000), 'value': 7},
  {'time': (54000, 68400), 'value': 7}]}

In [47]:
# check shapes for new links
# example: toll plaza at GGB and a new rail-to-street link

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin([8063198, 8063199, 26067378])][["id", "geometry"]]

,id,geometry
1634794,d01b27cabff869e0bf4f7187ee77d6de,"LINESTRING (-122.48221 37.83307, -122.48175 37..."
1634795,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
1634796,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [48]:
step5_scenario.road_net.shapes_df[
    step5_scenario.road_net.shapes_df.id.isin(["b46d3b9a4683bfe26b97e2a66181d8dd", "ba0e166e3a9f6c1cc8d7e331415769b2"])][
        ["id", "geometry"]]

,id,geometry
869991,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
869992,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [49]:
step5_scenario.transit_net.feed.shapes.shape_id.nunique()

1592

In [50]:
step5_scenario.transit_net.feed.shapes[
    step5_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [51]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.A.isin([2579949, 2607569])][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
514595,2579949,2535246,residential,0
842601,2579949,2604856,residential,1
846406,2607569,2586861,residential,1
1009927,2607569,2546185,residential,1
1077889,2607569,2541364,residential,0
1645033,2579949,300963,taz,1
1645951,2607569,300963,taz,1
1820125,2579949,318103,maz,1
1820126,2579949,327248,maz,1
1820127,2579949,328044,maz,1


In [52]:
# check Service Road Dumbarton Bridge Bypass is not assignable
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 3152465][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
778002,2552715,2540313,motorway_link,0


In [53]:
# check should be 3 lanes

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 2217343][["A", "B", "roadway","assignable", "lanes"]]

,A,B,roadway,assignable,lanes
690625,2153360,2071468,motorway,1,"{'default': 3, 'timeofday': [{'time': (21600, ..."


In [54]:
# should be ramps
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
        [3031886,3006561,3046803,3101610,3011518,3024387,4018032])][["A", "B", "roadway","assignable", "ft", "walk_access"]]

,A,B,roadway,assignable,ft,walk_access
33757,2506600,2542798,motorway_link,1,3,1
59112,2511462,2523557,motorway_link,1,3,1
123885,3017507,3058438,motorway_link,1,3,1
124668,2523557,2549665,motorway_link,1,3,1
162620,2501507,2506600,motorway_link,1,3,1
238525,2542798,2518014,motorway_link,1,3,1
518512,2580420,2511462,motorway_link,1,3,1


In [55]:
# zone id should be "2" (ac transit), "1" (dumbarton union city), "2" (ac transit dumbarton), "1" (samtrans) , "1" (samtrans)

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["101523872", "4694", "4669", "102036044", "4187"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
4130,Transbay Temp Terminal,37.789707,-122.393626,2,ACTransit_2015_8_14,50751,NaN,place_ETSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4187,3955000020,459ceb872e72e49fd4041a78884935e2,1021597
4611,Union City BART,37.590360,-122.017465,1,ACTransit_2015_8_14,50781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4669,1025879893,84c3f377f9f0065bf14be3af92202d0d,2509987
4636,Stanford Oval,37.429272,-122.168925,2,ACTransit_2015_8_14,55432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4694,522171022,6d2aeffcbad07f0873a268bea42838f3,2149155
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872


In [56]:
# check zone_id is removed for gg ferry stops

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["7706", "7707", "7708"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
7603,San Franciso Ferry Terminal,37.796169,-122.391617,NaN,GGTransit_2015_9_3,43000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7706,NaN,NaN,1027623
7604,Sausalito Ferry Terminal,37.856403,-122.478249,NaN,GGTransit_2015_9_3,43003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7707,NaN,NaN,5026533
7605,Larkspur Ferry Terminal,37.945316,-122.508736,NaN,GGTransit_2015_9_3,43004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7708,NaN,NaN,5026532


In [57]:
# check zone_id is updated for fairfield

step5_scenario.transit_net.feed.stops[step5_scenario.transit_net.feed.stops.stop_id.isin(
    ["6817", "6972"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
6741,El Cerrito Del Norte BART,37.924894,-122.317048,671,Fairfield_2015_10_14,400,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6972,1781110705,e235e2f09ba74bdbe6b6abb3e44fc47e,3060051
6779,Main Street and Lotz Way,38.243081,-122.040478,663,Fairfield_2015_10_14,172,0.0,NaN,Far-Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6817,62481309,52b70c286cb2792528a436f5736b33f7,3512083


In [58]:
# check USECLASS on I-80 WB to Bay Bridge Toll Plaza

step5_scenario.road_net.links_df[step5_scenario.road_net.links_df['model_link_id'].isin([3061568, 3149471])].useclass

314471    {'default': 0, 'timeofday': [{'time': (21600, ...
762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [59]:
# check zone-id for new stops on SamTrans
step5_scenario.transit_net.feed.stops[
    step5_scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [102036044,101511493,101516920,101525498,101516256,101513094,101513976,101528479,101529690,101529454,101519622,101513122,101514054,101543182,101512385,101534620,101503246,101537667,101538028,101526122,101519051,101503077,101508313,101508126,101537888,101519350,101516319,101509775,101536669])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22310,NaN,37.472389,-122.216100,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101511493,NaN,NaN,1511493
22311,NaN,37.483308,-122.231206,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516920,NaN,NaN,1516920
22312,NaN,37.485404,-122.233938,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101525498,NaN,NaN,1525498
22313,NaN,37.489469,-122.238568,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516256,NaN,NaN,1516256
22314,NaN,37.503227,-122.255333,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513094,NaN,NaN,1513094
22315,NaN,37.507359,-122.260283,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513976,NaN,NaN,1513976
22316,NaN,37.526752,-122.283581,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101528479,NaN,NaN,1528479
22317,NaN,37.540371,-122.301416,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529690,NaN,NaN,1529690
22318,NaN,37.559804,-122.322379,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529454,NaN,NaN,1529454


In [60]:
# check zone-id for new stops on SamTrans
step5_scenario.transit_net.feed.stops[
    step5_scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [101523872,101545076,101503345,101519909,101504028,101514714,101506746])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872
22339,NaN,37.682409,-122.487089,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101545076,NaN,NaN,1545076
22340,NaN,37.697643,-122.483092,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101503345,NaN,NaN,1503345
22341,NaN,37.701361,-122.484757,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101519909,NaN,NaN,1519909
22342,NaN,37.702417,-122.481156,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101504028,NaN,NaN,1504028
22343,NaN,37.704069,-122.475537,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101514714,NaN,NaN,1514714
22344,NaN,37.671171,-122.464817,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101506746,NaN,NaN,1506746


In [61]:
# check San Mateo bridge toll plaza
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
    [3315463,3315462, 3024600])
].lanes.tolist()

[{'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 7,
  'timeofday': [{'time': (21600, 36000), 'value': 5},
   {'time': (54000, 68400), 'value': 5}]}]

In [62]:
# check Bay Bridge west pan
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id.isin(
    [38545])
]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
818708,"['nan', '']",False,True,dc196b0fb1ac8053a741cd53df6ff64d,5,"['50 mph', '']","['San Francisco – Oakland Bay Bridge', '']","[True, True]","['I 80', '']",motorway,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Make Travel Model Networks

In [63]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = step5_scenario.road_net, 
    parameters = parameters)

2021-09-30 14:31:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:31:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:31:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:31:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:31:17, INFO: Filling nan for network from network wrangler
2021-09-30 14:31:17, INFO: Filling nan for network from network wrangler
2021-09-30 14:31:47, INFO: Converting variable type to mtc standard
2021-09-30 14:31:47, INFO: Converting variable type to mtc standard


In [64]:
# Vallejo Ferry to TAP

model_net.links_df[model_net.links_df.B == 3541176][["A", "roadway", "county", "rail_only", "model_link_id"]]

,A,roadway,county,rail_only,model_link_id
191843,3512789,footway,Solano,0,5013621
1634815,1028039,,Contra Costa,1,26067374
1634817,1028040,,Contra Costa,1,26067376
2235714,590188,tap,Solano,0,5157453


In [65]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2235780, 44)
2235780
(694964, 12)
694964
(1170510, 6)
1170510


In [66]:
# update farezone due to AC Transit, Fairfield, gg ferries Fare change

model_net = mtc.calculate_farezone(
    roadway_network = model_net,
    transit_network = step5_scenario.transit_net,
    parameters = parameters,
    network_variable = 'farezone',
    overwrite = True,
)

model_net.nodes_df.farezone.value_counts()

2021-09-30 14:36:46, INFO: Determining farezone
2021-09-30 14:36:46, INFO: Determining farezone
2021-09-30 14:36:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:36:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:36:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:36:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:36:46, INFO: Overwriting existing Variable 'farezone' already in network
2021-09-30 14:36:46, INFO: Overwriting existing Variable 'farezone' already in network
2021-09-30 14:36:46, INFO: Calculating and adding roadway network variable: farezone
2021-09-30 14:36:46, INFO: Calculating and adding roadway network variable: farezone
2021-09-30 14:36:47, INFO: Finished determining variable: farezone
2021-09-30 14:36:47, INFO: Finished determining variable: farezone


7.0      4210
92.0     1551
85.0      225
74.0      208
105.0     179
         ... 
71.0        1
18.0        1
73.0        1
75.0        1
76.0        1
Name: farezone, Length: 125, dtype: int64

In [67]:
model_net.nodes_df[model_net.nodes_df.model_node_id.isin([1523872,1545076,1503345,1519909,1504028,1514714,1506746])].farezone

38492     92.0
46931     92.0
77849     92.0
169823    92.0
229436    92.0
275080    92.0
512571    92.0
Name: farezone, dtype: float64

In [68]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-09-30 14:36:52, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-09-30 14:36:52, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-09-30 14:36:52, INFO: Creating managed lane network.
2021-09-30 14:36:52, INFO: Creating managed lane network.
2021-09-30 14:36:52, INFO: Creating network with duplicated managed lanes
2021-09-30 14:36:52, INFO: Creating network with duplicated managed lanes


C:\Users\wangs1\AppData\Roaming\Python\Python37\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\wangs1\AppData\Roaming\Python\Python37\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,

2021-09-30 14:50:21, INFO: Determining cntype
2021-09-30 14:50:21, INFO: Determining cntype
2021-09-30 14:50:21, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:50:21, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:50:21, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:50:21, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:50:21, INFO: Calculating and adding roadway network variable: cntype
2021-09-30 14:50:21, INFO: Calculating and adding roadway network variable: cntype
2021-09-30 14:51:58, INFO: Finished determining variable: cntype
2021-09-30 14:51:58, INFO: Finished determining variable: cntype
2021-09-30 14:51:58, INFO: Determining transit
2021-09-30 14:51:58, INFO: Determining transit
2021-09-30 14:51:58, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 14:51:58, INFO: Lasso base directory set a

In [69]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2240283, 55)
2240283
(696500, 14)
696500
(1175025, 7)
1175025


In [70]:
model_net.links_df.rail_only.value_counts()

0    2239507
1        776
Name: rail_only, dtype: int64

In [71]:
model_net.links_df[(model_net.links_df.bus_only == 1)].cntype.unique()

array(['TANA'], dtype=object)

In [72]:
model_net.links_df.assignable.value_counts()

0    1282943
1     957340
Name: assignable, dtype: int64

In [73]:
model_net.links_df[model_net.links_df.cntype == "TAZ"].assignable.value_counts()

1    32986
Name: assignable, dtype: int64

In [74]:
model_net.links_df[model_net.links_df.cntype == "MAZ"].assignable.value_counts()

1    531534
Name: assignable, dtype: int64

In [75]:
model_net.links_df[model_net.links_df.A == 2601773].roadway

1313           motorway
4318          ml_access
803801    motorway_link
Name: roadway, dtype: object

In [76]:
model_net.links_df[model_net.links_df.model_link_id == 3152465].shstReferenceId

782651    e5742b085ba3dcb41da0c499cecc33bc
Name: shstReferenceId, dtype: object

In [77]:
# AC Transit node farezone
model_net.nodes_df[model_net.nodes_df.model_node_id == 1011480].farezone
model_net.nodes_df.farezone.max()

125

In [78]:
model_net.links_df.cntype.value_counts()

TANA     1006677
PED       631872
MAZ       531534
TAP        36438
TAZ        32986
CRAIL        776
Name: cntype, dtype: int64

In [79]:
model_net.nodes_mtc_df[[c for c in model_net.nodes_mtc_df.columns if c in parameters.output_variables]].columns

Index(['osm_node_id', 'county', 'drive_access', 'walk_access', 'bike_access',
       'N', 'rail_only', 'X', 'Y', 'geometry', 'farezone', 'tap_id'],
      dtype='object')

In [80]:
model_net.links_mtc_df[[c for c in model_net.links_mtc_df.columns if c in parameters.output_variables]].columns

Index(['A', 'B', 'assignable', 'bike_access', 'bus_only', 'county', 'distance',
       'drive_access', 'ft', 'managed', 'model_link_id', 'name', 'rail_only',
       'segment_id', 'shstGeometryId', 'tollbooth', 'tollseg', 'transit',
       'walk_access', 'cntype', 'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM',
       'lanes_EV', 'useclass_EA', 'useclass_AM', 'useclass_MD', 'useclass_PM',
       'useclass_EV', 'geometry'],
      dtype='object')

In [81]:
model_net.links_df.county.value_counts()

Santa Clara      660906
Alameda          443746
Contra Costa     326804
San Mateo        203905
Sonoma           178851
Solano           157493
San Francisco    133512
Marin             84972
Napa              47068
                   2998
External             28
Name: county, dtype: int64

In [82]:
model_net.nodes_df.county.value_counts()

Santa Clara      203748
Alameda          137128
Contra Costa     104665
San Mateo         61887
Sonoma            59970
Solano            50710
San Francisco     33406
Marin             28439
Napa              14990
                   1536
External             21
Name: county, dtype: int64

In [83]:
# add county

model_net = mtc.calculate_county(
    roadway_network = model_net,
    parameters = parameters,
    network_variable = 'county'
)

2021-09-30 15:24:05, INFO: County Variable 'county' already in network, calculating for missing ones
2021-09-30 15:24:05, INFO: County Variable 'county' already in network, calculating for missing ones
2021-09-30 15:24:05, INFO: Adding roadway network variable for county using a spatial join with: Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\county\county.shp
2021-09-30 15:24:05, INFO: Adding roadway network variable for county using a spatial join with: Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\county\county.shp


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2759: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid


2021-09-30 15:27:48, INFO: Finished Calculating link county variable: county
2021-09-30 15:27:48, INFO: Finished Calculating link county variable: county
2021-09-30 15:28:43, INFO: Finished Calculating node county variable: county
2021-09-30 15:28:43, INFO: Finished Calculating node county variable: county


In [84]:
model_net.links_mtc_df['county'] = model_net.links_df['county']
model_net.nodes_mtc_df['county'] = model_net.nodes_df['county']

In [85]:
model_net.links_mtc_df.county.value_counts()

Santa Clara      662698
Alameda          444167
Contra Costa     327165
San Mateo        203945
Sonoma           178969
Solano           157577
San Francisco    133512
Marin             85148
Napa              47068
External             34
Name: county, dtype: int64

In [86]:
model_net.nodes_mtc_df.county.value_counts()

Santa Clara      204654
Alameda          137350
Contra Costa     104856
San Mateo         61910
Sonoma            60031
Solano            50751
San Francisco     33407
Marin             28530
Napa              14990
External             21
Name: county, dtype: int64

# Write to Disk

In [87]:
model_net.links_mtc_df['name'] = model_net.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))

In [88]:
model_net.links_mtc_df['name'].str.len().max()

85

In [89]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", "cntype", "lanes_AM", "assignable", "useclass_AM", 'name', 'tollbooth'],
    node_output_variables = ["model_node_id", "N", "geometry", "farezone", "tap_id"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-09-30 15:32:17, INFO: Writing Network as Shapefile
2021-09-30 15:32:17, INFO: Writing Network as Shapefile
2021-09-30 15:32:17, INFO: Renaming DBF Node Variables
2021-09-30 15:32:17, INFO: Renaming DBF Node Variables
2021-09-30 15:32:17, INFO: Renaming variables so that they are DBF-safe
2021-09-30 15:32:17, INFO: Renaming variables so that they are DBF-safe
2021-09-30 15:32:30, INFO: Renaming DBF Link Variables
2021-09-30 15:32:30, INFO: Renaming DBF Link Variables
2021-09-30 15:32:30, INFO: Renaming variables so that they are DBF-safe
2021-09-30 15:32:30, INFO: Renaming variables so that they are DBF-safe
2021-09-30 15:32:31, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\nodes.shp
2021-09-30 15:32:31, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\nodes.shp
2021-09-30 15:33:31, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

z:\data\users\sijia\mtc\github\lasso\lasso\roadway.py:1565: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  links_dbf_df.to_file(output_link_shp)


In [90]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-09-30 15:38:15, INFO: Starting fixed width conversion
2021-09-30 15:38:15, INFO: Starting fixed width conversion
2021-09-30 15:59:07, INFO: Writing out link database
2021-09-30 15:59:07, INFO: Writing out link database
2021-09-30 15:59:25, INFO: Writing out link header and width ----
2021-09-30 15:59:25, INFO: Writing out link header and width ----
2021-09-30 15:59:25, INFO: Starting fixed width conversion
2021-09-30 15:59:25, INFO: Starting fixed width conversion
2021-09-30 16:01:40, INFO: Writing out node database
2021-09-30 16:01:40, INFO: Writing out node database
2021-09-30 16:01:43, INFO: Writing out node header and width
2021-09-30 16:01:43, INFO: Writing out node header and width


In [91]:
%%capture
os.chdir(output_dir)
!make_cube.bat

## As transit line file (for CUBE)

In [92]:
mtc.write_cube_fare_files(
    roadway_network=model_net,
    transit_network=step5_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [93]:
standard_transit_net = StandardTransit.fromTransitNetwork(step5_scenario.transit_net, parameters = parameters)

2021-09-30 16:03:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 16:03:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 16:03:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-30 16:03:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [94]:
standard_transit_net.feed.routes[standard_transit_net.feed.routes.route_id=="712"]

,route_id_original,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,agency_raw_name,route_desc,route_url,route_sort_order,min_headway_minutes,eligibility_restricted,continuous_pickup,continuous_drop_off,route_id
701,brisbane,alliance,NaN,Brisbane-Bayshore Caltrain,3,NaN,NaN,commuteDOTorg_GTFSImportExport_20160127_final_mj,NaN,http://commute.org/index.php?option=com_conten...,NaN,NaN,NaN,NaN,NaN,712


In [95]:
standard_transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 16 columns):
route_id_original         721 non-null object
agency_id                 624 non-null object
route_short_name          666 non-null object
route_long_name           699 non-null object
route_type                721 non-null int64
route_color               260 non-null object
route_text_color          243 non-null object
agency_raw_name           721 non-null object
route_desc                14 non-null object
route_url                 236 non-null object
route_sort_order          95 non-null object
min_headway_minutes       32 non-null object
eligibility_restricted    4 non-null object
continuous_pickup         4 non-null object
continuous_drop_off       4 non-null object
route_id                  721 non-null object
dtypes: int64(1), object(15)
memory usage: 90.2+ KB


In [96]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-09-30 16:03:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:03:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:03:37, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:03:37, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:17, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:17, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:25, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:25, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:27, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:27, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:30, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:05:30, INFO: Converting GTFS Standard Pr

In [97]:
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-09-30 16:11:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-09-30 16:11:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


# write EMME
## this part writes out EMME networks, which require EMME license

In [106]:
from lasso import emme

In [107]:
emme.create_emme_network(
    links_df=model_net.links_mtc_df,
    nodes_df=model_net.nodes_mtc_df,
    transit_network=standard_transit_net,
    name="version 12",
    path=output_dir,
    write_drive_network = True,
    write_maz_active_modes_network = True,
    write_tap_transit_network = True,
    parameters=parameters,
    subregion_boundary_file= os.path.join(lasso_dir, 'mtc_data', 'emme', 'subregion_boundary_for_active_modes.shp') ,
    subregion_boundary_id_variable = 'subregion'
)

2021-09-30 17:01:24, INFO: Converting CRS of input network to epsg 4326
2021-09-30 17:01:24, INFO: Converting CRS of input network to epsg 4326
2021-09-30 17:02:59, INFO: Creating shapes for backward directions on two-way links
2021-09-30 17:02:59, INFO: Creating shapes for backward directions on two-way links
2021-09-30 17:04:57, INFO: Converting geometry into wkt geometry
2021-09-30 17:04:57, INFO: Converting geometry into wkt geometry


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\computation\expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


d:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_12\emme_drive_network\database\emmebank
2021-09-30 17:08:53, INFO: Spliting activate mode network into subnetworks using Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\emme\subregion_boundary_for_active_modes.shp
2021-09-30 17:08:53, INFO: Spliting activate mode network into subnetworks using Z:/Data/Users/Sijia/MTC/github/Lasso\mtc_data\emme\subregion_boundary_for_active_modes.shp
d:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_12\emme_maz_active_modes_network_subregion_south\database\emmebank
d:\github\lfs-cleaning\travel-model-two-networks\data\processed\version_12\emme_maz_active_modes_network_subregion_north\database\emmebank
2021-09-30 17:24:38, INFO: Converting GTFS Standard Properties to MTC's Emme Standard
2021-09-30 17:24:38, INFO: Converting GTFS Standard Properties to MTC's Emme Standard
2021-09-30 17:24:38, INFO: Reading faresystem from D:/github/lfs-cleaning/travel-model-two-net

## As pickle

In [98]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_12.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [99]:
step5_scenario.road_net.write(path = output_dir, filename = "v12")
step5_scenario.transit_net.write(path = output_dir, filename = "v12")

2021-09-30 16:30:55, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12
2021-09-30 16:30:55, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12


In [100]:
# save out link feather
# to_feather() does not like mixed type

links_feather = step5_scenario.road_net.links_df.copy()
links_feather.drop("geometry", axis = 1, inplace = True)

for c in links_feather.columns:
    if c in parameters.int_col:
        links_feather[c] = links_feather[c].astype(int)
    else:
        links_feather[c] = links_feather[c].astype(str)
        
links_feather.to_feather(path = os.path.join(output_dir, "v12_links.feather"))

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\ipykernel_launcher.py:13: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  del sys.path[0]


In [101]:
stop_node_id = step5_scenario.transit_net.feed.stops.model_node_id.tolist()

In [102]:
step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id.isin(stop_node_id)].walk_access.value_counts()

1    18215
Name: walk_access, dtype: int64

In [103]:
model_net.links_df[model_net.links_df.model_link_id == 3033292].assignable

175454    1
Name: assignable, dtype: int32

In [104]:
model_net.nodes_df.columns

Index(['index', 'osm_node_id', 'shst_node_id', 'county', 'drive_access',
       'walk_access', 'bike_access', 'model_node_id', 'rail_only', 'X', 'Y',
       'geometry', 'farezone', 'tap_id'],
      dtype='object')

In [105]:
step5_scenario.transit_net.feed.stops.model_node_id.nunique()

18215